In [ ]:
import numpy as np
import arlpy.signal as asig
from arlpy.plot import *
from unetpy import *

### Connect to the modem

In [ ]:
modem = UnetGateway('192.168.1.34')

In [ ]:
phy = modem.agent('phy')
modem.subscribe(phy)

### Set passband block count & signal power level

In [ ]:
fs = 96000
sigLen = 5e-3
phy.pbsblk = 45056
phy.signalPowerLevel = -10

In [ ]:
def flush_modem():
    while modem.receive(timeout=1000):
        pass

### Generater the signal to be transmitted

In [ ]:
tx = asig.cw(22000, sigLen, fs*2).tolist()

### Start baseband signal reception and transmit the signal

In [ ]:
flush_modem()
phy.npulses = 1
#phy.pulsedelay = 100
phy.pbscnt = 1
phy << org_arl_unet_bb.TxBasebandSignalReq(signal=tx, fc=0)
rx = modem.receive(org_arl_unet_bb.RxBasebandSignalNtf, 5000)
txntf = modem.receive(timeout=5000)
txTime = txntf.txTime
rxTime = rx.rxTime

### Plot the recorded signal & PSD

In [ ]:
plot(rx.signal, fs=rx.fs, maxpts=48000, hold=True)
txStart=(txTime-rxTime)/1e6
vlines(txStart, color='orange')

In [ ]:
psd(rx.signal, fs=fs, nfft=512, noverlap=None, window='hanning', color=None, style='solid', thickness=1, marker=None, filled=False, size=6, title=None, xlabel='Frequency (Hz)', ylabel='Power spectral density (dB/Hz)', xlim=None, ylim=None, width=None, height=None, hold=False, interactive=None)

In [ ]:
env = asig.envelope(rx.signal[])
threshold = 0.9
edge = np.flatnonzero((env[:-1] < threshold) & (env[1:] > threshold))+1

In [ ]:
plot(env, fs=fs, maxpts = 96000, hold=True)
#txStart=(txTime-rxTime)/1e6
rxStart=(edge[0]/fs)
vlines(txStart, hold=True, color='orange')
vlines(txStart+sigLen, color='purple')

In [ ]:
print ('Tx Start = ', txStart, 's')
print ('Rx Start = ', rxStart, 's')
travelTime = rxStart-txStart
print ('Round trip time = ', travelTime, 's')
soundSpeed = 60
print ('Range = ', soundSpeed * travelTime, 'm')

In [ ]:
modem.shutdown()